In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts

In [3]:
class NaiveBayes:
    def fit(self, x, y):
        n_samples, n_features = x.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # Calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            x_c = x[y == c]
            self._mean[idx, :] = x_c.mean(axis=0)
            self._var[idx, :] = x_c.var(axis=0)
            self._priors[idx] = x_c.shape[0] / float(n_samples)

    def predict(self, x):
        y_pred = [self._predict(sample) for sample in x]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # Calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            class_conditionals = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + class_conditionals
            posteriors.append(posterior)

        # Return class with the highest posterior probability
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

# Testing
if __name__ == "__main__":
    from sklearn.model_selection import train_test_split as tts
    from sklearn.metrics import confusion_matrix, classification_report
    import pandas as pd

    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

   

In [6]:
df = pd.read_csv("diabetes.csv")
df = df.sample(frac=1)
data = np.array(df)
x = data[:, 0:8]
y = data[:, 8]
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2)
nb = NaiveBayes()
nb.fit(x_train, y_train)
predictions = nb.predict(x_test)

print("NAIVE BAYES CLASSIFIER")
print("ACCURACY FOR TEST SET:", accuracy(y_test, predictions))

print("\nCONFUSION MATRIX")
cm = np.array(confusion_matrix(y_test, predictions, labels=[1, 0]))
confusion = pd.DataFrame(
        cm, index=["Has_diabetes", "Is_healthy"], columns=["predicted_diabetes", "predicted_healthy"]
    )

print("\n")
print(confusion, "\n\n")
print("CLASSIFICATION REPORT")
print(classification_report(y_test, predictions))


NAIVE BAYES CLASSIFIER
ACCURACY FOR TEST SET: 0.7792207792207793

CONFUSION MATRIX


              predicted_diabetes  predicted_healthy
Has_diabetes                  30                 20
Is_healthy                    14                 90 


CLASSIFICATION REPORT
              precision    recall  f1-score   support

         0.0       0.82      0.87      0.84       104
         1.0       0.68      0.60      0.64        50

    accuracy                           0.78       154
   macro avg       0.75      0.73      0.74       154
weighted avg       0.77      0.78      0.78       154

